## Data Collection

    - Scrape data from basketball-reference.com from the NBA seasons 2014-2021

In [ ]:
# Imports
import pandas as pd
import numpy as np
import requests
import csv
import os

### Advanced Team Stats 2014-2021

In [ ]:
# Scrape advanced team data 2014-2021

# Function to get HTML table from a basketball-reference.com url
def get_adv_team_data(year):
    
    # Initialize empty dataframe to store each year's advanced team statistics
    adv_stats = pd.DataFrame()
    
    html = requests.get(base_url + 'NBA_' + str(year) + '.html').content # Ex.: https://www.basketball-reference.com/leagues/NBA_2021.html
    df_list = pd.read_html(html) #get all html tables
    for df in df_list:
        if len(df.columns) == 31:
            adv_stats = df
            adv_stats.columns = ['Rank', 'Team', 'Age', "W", 'L', 'Pythagorean Wins', 'Pythagorean Losses',
                                'MOV', 'SOS', 'SRS', 'ORtg', 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', "TS%", '', 'OEFG%',
                                'OTOV%', 'OORB%', 'OFT/FGA', '', 'DEFG%', 'DTOV%', "DDRB%", 'DFT/FGA', '', 'Arena', 'Attend.',
                                'Attend./G']
            
            # Drop last row ('League Averages')
            adv_stats.drop(adv_stats.tail(1).index, inplace = True)
            # Drop columns with any empty values 
            adv_stats.dropna(axis = 1, inplace = True)
            
            return adv_stats

# Function to save csv file to current working directory
def save_table(df, directory, name):
    if not os.path.exists(current + '/' + directory + '/'):
        os.mkdir(current + '/' + directory + '/')
        
    df.to_csv(current + '/' + directory + '/' + str(name) + '.csv', index = False)

In [ ]:
# Scrape the advanced stats
base_url = 'https://www.basketball-reference.com/leagues/'

# Get current working directory
current = os.getcwd()

# Generate years list
years = [x for x in range(2014, 2022, 1)] 
    
# Scrape 2014-2021 stats
for year in years:
    stats = get_adv_team_data(year)
    save_table(stats, 'Advanced Stats', str(year) + 'Team_Stats')

### Individual Game Results 2014-2021

In [ ]:
# Scrape individual game result data 2014-2021

# Function to store game results of each month in variable game_results and return it
def get_results(year):
    
    # Initialize empty dataframe to store each month's results for a given year
    game_results = pd.DataFrame()
    
    # Iterate over months and extract html tables
    for month in months:
        url = base_url + 'NBA_' + str(year) + '_games-' + month + '.html'  # Example: base_url + NBA_2021_games-december.html'
        try: # check if url is valid
            html = requests.get(url).content
            df = pd.read_html(html)[0]
            game_results = game_results.append(df)
        except:
            pass
        
    return game_results

# Function to clean up table, remove NaN, rename columns, etc.
def clean_dataframe(df):
    
    # Rename columns
    df.columns = ['Date', '', 'Away', 'PTS', 'Home', 'PTS1', '', '', '', '']
    
    #Remove unnecessary columns/rows
    df = df.drop('', axis = 1)
    df = df[df.Date != 'Playoffs']
    df.dropna(inplace = True, axis = 0) # drop empty values - games postponed due to Covid-19
    
    # Generate new column 'ID' 
    id_list = [x for x in range(1, len(df)+1)]
    df.insert(loc = 0, column = 'ID', value = id_list)
    
    return df

# Function to split each game into two rows to represent each team (prep for future merge)
def split_games_in_table(df):
    
    # Convert points to numeric for comparison
    df[['PTS', 'PTS1']] = df[['PTS', 'PTS1']].apply(pd.to_numeric)

    # Initiliaze a new dataframe
    new_df = pd.DataFrame(columns = ['GameID', 'Date', 'Team', 'Points', 'Home/Away', 'Win'])

    # Iterate over each row of df
    for i in range(1,len(df)+1):
        
        # Locate rows by ID
        row = df.o[df['ID'] == i]

        # Set values for home team and away team based on the original dataset, determine target variable 'Win'
        new_row_away = {'GameID':i, 'Date':row['Date'].values[0], 'Team':row['Away'].values[0],
                        'Points':int(row['PTS'].values[0]),
                        'Home/Away':'Away','Win':int(row['PTS'].values[0]>row['PTS1'].values[0])}


        new_row_home = {'GameID':i, 'Date':row['Date'].values[0], 'Team':row['Home'].values[0],
                        'Points':int(row['PTS1'].values[0]),
                        'Home/Away':'Home','Win':int(row['PTS1'].values[0]>row['PTS'].values[0])}

    
        # Append the rows to the new dataframe
        new_df = new_df.append(new_row_away, ignore_index=True)
        new_df = new_df.append(new_row_home, ignore_index=True)
        
    return new_df

# Function to save csv file to current working directory
def save_dataframe(df, directory, name):
    
    # Check if directory exists, if not, create one 
    if not os.path.exists(current + '/' + directory + '/'):
        os.mkdir(current + '/' + directory + '/')
    
    # Write file to csv
    df.to_csv(current + '/' + directory + '/' + str(name) + '.csv', index = False)

In [ ]:
base_url = 'https://www.basketball-reference.com/leagues/'

# Get current working directory
current = os.getcwd()

# Create a list of months to append to base url'
months = ['september', 'october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august']

# Generate years list
years = [x for x in range(2014, 2022, 1)] 

# Scrape, clean, prep and save 2014-2021 stats
for year in years:
    stats = get_results(year)
    stats = clean_dataframe(stats)
    stats = split_games_in_table(stats)
    save_dataframe(stats, 'All Game Results', str(year) + '_all_NBA_games')


### Team Box Score Each Game 2014-2021

In [ ]:
# Scrape team box score of each game 2014-2021
base_url = 'https://www.basketball-reference.com/boxscores/'

# Dictionary mapping team name to abbreviation
team_name_to_short = {'Atlanta Hawks':'ATL',
'Boston Celtics':'BOS',
'Brooklyn Nets':'BRK',              
'Charlotte Hornets':'CHO',
'Charlotte Bobcats':'CHA',
'Chicago Bulls':'CHI',
'Cleveland Cavaliers':'CLE',
'Dallas Mavericks':'DAL',
'Denver Nuggets':'DEN',
'Detroit Pistons':'DET',
'Golden State Warriors':'GSW',
'Houston Rockets':'HOU',
'Indiana Pacers':'IND',
'Los Angeles Clippers':'LAC',
'Los Angeles Lakers':'LAL',
'Memphis Grizzlies':'MEM',
'Miami Heat':'MIA',
'Milwaukee Bucks':'MIL',
'Minnesota Timberwolves':'MIN',
'New Orleans Pelicans':'NOP',
'New York Knicks':'NYK',
'Oklahoma City Thunder':'OKC',
'Orlando Magic':'ORL',
'Philadelphia 76ers':'PHI',
'Phoenix Suns':'PHO',
'Portland Trail Blazers':'POR',
'Sacramento Kings':'SAC',
'San Antonio Spurs':'SAS',
'Toronto Raptors':'TOR',
'Utah Jazz':'UTA',
'Washington Wizards':'WAS'}

# Dictionary mapping month name to its numeric representation
month_to_num = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06', 'Jul':'07',
                    'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

# Function to convert nba_games date column to numbers
# Ex.: Tue, Oct 29, 2013 -> 20131029
def date_to_nums(date):
    date = date[5:]
    year = date.split(' ')[2]
    month = month_to_num[date[0:3]]
    day = date.split(' ')[1].replace(',','')
    if len(day) == 1:
        day = '0' + day

    return year+month+day

# Function to import csv from local file directory
def load_data(filename):
        return pd.read_csv(current + '/' + 'All Game Results/' + filename)

# Function that iterates over all games of given season, creates a url of a particualar game's statistics,
# scrapes the team's statistics of that game, appends it to a new dataset, and returns the final dataset
# after all iterations
def get_team_boxscores(season_games_df):
    
    # Initiliaze for progress 
    prev = 0
    
    # Initialize an empty dataset to collect team statistics
    team_boxscores = pd.DataFrame()

    # Loop over all games. Since the game is split into two rows (one for each team), range step is 2.
    for i in range(1, len(season_games_df), 2):
        
        # Collect date, home team for url creation
        date, home_team = season_games_df.iloc[i, 1], season_games_df.iloc[i, 2]

        # Ex. https://www.basketball-reference.com/boxscores/201310290IND.html
        url = base_url + date_to_nums(date) + '0' + team_name_to_short[home_team] + '.html'

        try: # check if url is valid
            html = requests.get(url).content
            data = pd.read_html(html)

            totals_away = pd.DataFrame(data[0].iloc[-1]).T #transposed
            totals_away.columns = ['', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
                          'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-']
            totals_away = totals_away.drop(['', 'MP', '+/-'], axis=1)

            # If a game had overtimes, the index of the boxscore changes. For OT - the index is 9. 
            # For 2OT - the index is 10. For 3OT - the index is 11. This for loop selects the appropriate table 
            # that represents team's box score.
            for k in range(8, 15):
                totals_home = pd.DataFrame(data[k].iloc[-1]).T

                if len(totals_home.columns) != 21:
                    continue

                totals_home.columns = ['', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
                              'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-']
                # Checks if MP (minutes played) is less than 26. If true, that boxscore represents OT, not the whole game. 
                if int(totals_home['MP']) < 26:
                    continue

                break

            totals_home = totals_home.drop(['', 'MP', '+/-'], axis=1)
            
            # Append the results to the dataset
            team_boxscores = team_boxscores.append(totals_away)
            team_boxscores = team_boxscores.append(totals_home)
            
            # Print progress
            new = int((round(i/len(season_games_df) * 100, 2)))
            if new > prev and new % 5 == 0:
                print(str(new) + '%')
                prev = new

        except Exception as e: 
            print('Exception in collecting box scores')
            # Check which url throws an error
            print(url)
            break          
            
    return team_boxscores
    
# Function to merge teams with their box score for each game in a given season and return merged dataset
def merge_games_with_boxscores(season_games_df, team_boxscores):
    
    # Create a common column 'temp' to merge on
    temp_list = [x for x in range(1, len(season_games_df)+1)]
    season_games_df.insert(loc = 0, column = 'temp', value = temp_list)
    team_boxscores.insert(loc = 0, column = 'temp', value = temp_list)
    result = pd.merge(season_games_df, team_boxscores, on = ['temp'])
    # Drop temp column
    result.drop('temp', axis = 1, inplace = True)
    
    return result

# Function to print unique values in a list. Used for verifying merged dataset
def unique(list1):
 
    # intilize a null list
    unique_list = []
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
            
    # print list
    for x in unique_list:
        print(x)

# Function to verify that datasets were merged correctly by comparing the points of two datasets
def verify_merged_data(merged_data):
    
    # Initiliaze empty list
    verify_list = []
    
    # Convert 'PTS' to numeric values
    merged_data[['PTS']] = merged_data[['PTS']].apply(pd.to_numeric)
    
    for x in range(0, len(result)):
        # Boolean that indicates if team was matched with proper boxscore by comparing the points
        # of two datasets for each row
        verify = result['Points'][x] == result['PTS'][x]
        
        # Add boolean to the list if does not exists already
        if verify not in verify_list:
            verify_list.append(verify)
            
    # Drop one of the points value from merged dataset
    merged_data.drop('Points', axis = 1, inplace = True)
            
    # If verify_list length is less than 2 (only 1 verify value in the list - True), datasets were merged correctly.
    if len(verify_list) < 2 and verify_list[0] == True:
        print('Team matched with boxscore correctly')
        return True
        
    else:
        print('Team did NOT match with boxscore correctly')
        return False

# Function to save csv file to current working directory
def save_dataframe(df, directory, name):
    
    # Check if directory exists, if not, create one 
    if not os.path.exists(current + '/' + directory + '/'):
        os.mkdir(current + '/' + directory + '/')
    
    # Write file to csv
    df.to_csv(current + '/' + directory + '/' + str(name) + '.csv', index = False)

In [ ]:
# Get current directory
current = os.getcwd()

# Collect the names of csv files containing all games data
files = os.listdir(current + '/All Game Results/' )

# For each season data
for file in files:
    # Load data
    game_results = load_data(file)
    print('Done loading')
    print(file)
    
    # Get box scores of that season
    team_boxscores = get_team_boxscores(game_results)
    print('Done loading boxscores')
    
    # Merge datasets
    merged = merge_games_with_boxscores(game_results, team_boxscores)
    print('Done merging')
    
    # Verify the merge
    verify = verify_merged_data(merged)
    print('Done verifying')
    
    # If merged correctly, save data
    if verify:
        save_dataframe(merged, 'Merged NBA data', file)